In [4]:
# SibSp + Parch - точно размер семьи
# "Survived": "Выжил", "Name": "Имя", "Sex": "Пол", "Age": "Возраст" - индивидуальные характеристики
# + "Pclass": "Класс билета" и "Fare": "Стоимость билета" - социальный статус, тоже индивидуальная х-ка
# Женщины без детей выживают лучше, чем с детьми; Мужчины без детей практически не выживают; Ситуация чуть уравнивается, если есть больше 0 детей или родителей на борту
# близкая к 50% выживаемость у мужчин только в 1 классе; Женщины в 1 классе практически все выжили, во 2 аналогично, в 3 50/50
# Проверить порт посадки; Для женщин: C - средний по численности, статистика выживаемости лучшая из 3 портов; следом идет Q по выживаемости, но народу там меньше всего; после идет S с худшей статистикой выживаемости и самой большой численностью
# Проверить порт посадки; Доля мужчин: C - средняя численность, лучшая выживаемость, средняя численность; Q - малая численность, худшая выживаемость, S - наибольшая численность, средняя выживаемость
# Можно попробовать взять рандомную семью с 4+ членами и глянуть на них
# cемья Fortune, есть женщины, родители, братья/сестры, есть отец? (или кто это), но в данных есть 1 супруг, по номеру билета не ищется, по фамилии тоже, по каюте тоже
# Будем считать что данных по ней нет, потому что по Quick кол-во супругов 0, хотя это женщина с 2 y.o. ребенком
# по Fortune, 2/2 по гендеру, все женщины выжили, мужчины нет. Возраст мужчин - 19, 64, женщин - 23, 24. 1 класс, каюты одинаковые, порт посадки S (для женщин: худшей статистикой выживаемости и самой большой численностью, мужчин: наибольшая численность, средняя выживаемость)
# Как правило, мужчин выжило гораздо меньше женщин, возраст тут роли не играет. Касаемо порта посадки, странно что женщины все выжили, мужчины нет.
# Надо взять прям гигансткую семейку теперь из 3 класса и глянуть что с ней случилось. Если это и Титаник - то весь 3 класс откис потому, что шлюпки заняла элита
# А одного из самых знаменитых актеров отправили дрейфовать в холодную воду
# 8, Sage, нормально
# в данных 8 сестер/братьев, 7 по факту. Все умерли, 3 класс, порт тоже S
# ---Исходя из данных, выходит что порт S, 3 класс билета, мужчина - априори умирает почти всегда, тогда как женщина, 1 класс, C поррт с ребенком почти всегда выживает
# ---большая семья, порт S, 3 класс билета, нет выживших (семьи Sage, Andersson, Goodwin)
# ---Женщина, ребенок, порт C, 1 класс - почти гарантия выжить
# Исключения - пока можно допустить прагматичность стариков, они уступали места другим.
# дети до 12 лет вкл - почти 50/50 умирали независимо от пола
# ---дети до 12 лет 1 класс - выживают 3/4
# ---дети до 12 лет 3 класс - выживают 1/5
# ---дети до 12 лет 2 класс - выживают 1 (все)
# столбчатый какую то херню показал
# ---Дети (<= 18) элиты (1 класс билета) выживали почти в 90% случаев

# 1. если Возраст <= 12 -> Выжил = 1
# 2. если Возраст <= 18 и Класс билета == 1 -> Выжил = 1
# 3. если "Количество братьев/сестер/супругов на борту" + "Количество родителей/детей на борту" >= 5 и Класс билета == 3 -> Выжил = 0
# 4. если Пол == 'female':
#     - если Класс билета == 1 или Класс билета == 2 -> Выжил = 1
#     - если Класс билета == 3:
#         - если Порт посадки == 'S' -> Выжил = 0
#         - иначе -> Выжил = 1
# 5. если Пол == 'male':
#     - если Класс билета == 1:
#         - если Возраст < 15 или Порт посадки == 'C' -> Выжил = 1
#         - иначе -> Выжил = 0
#     - иначе -> Выжил = 0

In [6]:
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv("../.env")

df = pd.read_csv(os.getenv("LOCAL_LESSON6_DS"))

df = df.rename(columns = { 
    "PassengerId": "Идентификатор пассажира",
    "Survived": "Выжил",
    "Pclass": "Класс билета", 
    "Name": "Имя",
    "Sex": "Пол",
    "Age": "Возраст",
    "SibSp": "Количество братьев/сестер/супругов на борту",
    "Parch": "Количество родителей/детей на борту",
    "Ticket": "Номер билета",
    "Fare": "Стоимость билета",
    "Cabin": "Номер каюты",
    "Embarked": "Порт посадки"
})

df['Возраст'] = df['Возраст'].fillna(df['Возраст'].median()) # проверить какая точность

def titan(row):
    if row['Возраст'] <= 12:
        return 1
    
    if row['Возраст'] <= 18 and row['Класс билета'] == 1:
        return 1
    
    if (row['Количество братьев/сестер/супругов на борту'] + row['Количество родителей/детей на борту'] >= 5) and row['Класс билета'] == 3:
        return 0
    
    if row['Пол'] == 'female':
        if row['Класс билета'] in [1, 2]:
            return 1
        else:
            return 0 if row['Порт посадки'] == 'S' else 1
    
    else:
        if row['Класс билета'] == 1:
            return 1 if row['Возраст'] < 15 or row['Порт посадки'] == 'C' else 0
        else:
            return 0
        
df['Прогноз'] = df.apply(titan, axis=1)

df.head(10)

,Идентификатор пассажира,Выжил,Класс билета,Имя,Пол,Возраст,Количество братьев/сестер/супругов на борту,Количество родителей/детей на борту,Номер билета,Стоимость билета,Номер каюты,Порт посадки,Прогноз
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q,0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,1
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,1


In [7]:
df[df['Возраст'].isna()]['Возраст'].value_counts(dropna=False)

Series([], Name: count, dtype: int64)

In [9]:
df['Верно'] = (df['Выжил'] == df['Прогноз']).astype(int)

accuracy = df['Верно'].sum()/len(df)

print(accuracy)

0.8035914702581369


In [10]:
(df['Верно'] == df['Прогноз']).mean()

np.float64(0.3838383838383838)

In [50]:
TP = df[(df['Верно'] == 1) & (df['Выжил'] == 1)]['Выжил'].sum()

In [41]:
df.query('Выжил == 1 & Выжил == Прогноз')['Выжил'].sum()

np.int64(244)

In [52]:
FP = df[(df['Выжил'] == 0) & (df['Прогноз'] == 1)]['Прогноз'].sum()

In [49]:
df[(df['Выжил'] == 1) & (df['Прогноз'] == 1)]['Выжил'].sum()/(df.query('Выжил == 1 & Прогноз == 1')['Выжил'].sum() + df[(df['Выжил'] == 0) & (df['Прогноз'] == 1)]['Прогноз'].sum())

np.float64(0.7601246105919003)

In [55]:
precision = TP/(TP + FP)
print(precision)

0.7601246105919003


In [56]:
FN = df[(df['Выжил'] == 1) & (df['Прогноз'] == 0)]['Выжил'].sum()

In [59]:
recall = TP/(TP + FN)
print(recall)

0.7134502923976608


In [60]:
F1 = 2 * ((precision * recall)/(precision + recall))
print(F1)

0.7360482654600301
